In [1]:
"""The idea is to put any math utility routines that we need to write in this module"""
import numpy as np
import scipy.integrate as integrate
import  math

In [100]:
def integral(f, xmin, xmax, nsteps, xminguess = 0, xmaxguess = 0, eBound = 0, maxIter = 10):
    """
    Name: 
        integrate
    
    Purpose:
        Integrate 1D functions using specified gridpoints, and optionally running until convergence to within a percentage, eBounds
    
    Explanation: 
        Integrate using a basic right-handed Riemann sum. 
    
    Calling Seuqence:
        integrator, f, xmin, xmax, nsteps, F
        
    Input/Output:
        f - The function to be integrated. Must return a float.
        xmin - Lower bound of integration
        xmax - Upper bound of integration
        nsteps - The number of points to use for computing the integral, as a starting point
        eBounds - The allowed percent error. Must be supplied if either bound is infinite
        xminguess - a guess for the minimum xvalue, if -inf. Must be supplied if xmax is infinite
        xmaxguess - a guess for the maximum xvalue, if inf. Must be supplied if either bound is infinite
        integral, error - A list containing the approximate value of the integral and its error, 
        or if no error bound is specified then just the appproximate value
    
    Limitations: If any gridpoints fall on a singularity, the routine will fail. 
    If there is a divergence not directly evaluated, the integral will fail to converge
    and we will get a nonsense answer. 
        
        
    
    """
    if (xmax == math.inf) & (xmaxguess == 0):
        return "Supply xmaxguess"
    if (xmin == -math.inf) & (xminguess == 0):
        return "Supply xminguess"
    if ((xmax == math.inf) | (xmin == -math.inf)) & (eBound == 0):
        return "Supply eBound"
    if eBound != 0:
        #The idea is that the contribution from taking out the integral further should be restricted to a higher decimal place 
        #each time. Of course this is not rigorously true (probably we need something about the function being monotonic), 
        #but it should be a useful heuristic. Note that the guesses must be "reasonable" and convergence is not guaranteed. 
        error = 1
        if xmax == math.inf:
            xmax = xmaxguess
            if xmin == -math.inf:
                xmin = xminguess
            n = 0
            
            while (error > (.1 * eBound)) & (n <= maxIter):
                xmax = xmax * 10
                nsteps = nsteps * 10
                xgrid = np.linspace(xmin, xmax, nsteps+1)
                ygrid = f(xgrid) 
                errorCont = np.absolute( f(xgrid[-2]) * (xgrid[-1] - xgrid[-2]))
                error = errorCont / integrate.simps(ygrid, xgrid)
                n += 1        
        if xmin == -math.inf:
            xmin = xminguess
            xmax = xmaxguess
            n = 0
            
            while ((errorCont > .1 * eBound)) & (n <= maxIter):
                xmax = xmax * 10
                xgrid = np.linspace(xmin, xmax, nsteps+1)
                ygrid = f(xgrid) 
                errorCont = np.absolute( f(xgrid[-2]) * (xgrid[-1] - xgrid[-2]))   
                error = errorCont / integrate.simps(ygrid, xgrid)
                n += 1
        if n == maxIter:
            return "Max iterations reached. Integral may not converge"
        #If we had infinite bounds, these next two lines do nothing. 
        xgrid = np.linspace(xmin, xmax, nsteps+1)
        ygrid = f(xgrid)        
        n = 0
        error = 1
        integral = 0
        while (error > eBound) & (n <= maxIter):
            nsteps = nsteps * 10
            xgrid = np.linspace(xmin, xmax, nsteps+1)
            ygrid = f(xgrid) 
            integralNew = integrate.simps(ygrid, xgrid)
            error = np.absolute((integral - integralNew)/integralNew)
            integral = integralNew
            n += 1
        if n == maxIter:
            return "Max iterations reached. Integral may not converge"
        return integral, error        

        
    else:
        xgrid = np.linspace(xmin, xmax, nsteps+1)
        ygrid = f(xgrid)
        integral = integrate.simps(ygrid, xgrid)
    return integral

        

In [101]:
#integral(lambda x: np.exp(-x), 0, 10, 10)

1.0049115556739088